In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, dcc, html

In [2]:
# Load Data
assessments = pd.read_pickle('../data/school_based/assessments_clean.pkl')

# Load Shape File
shape_file = '../data/Test Data/Geocode - School/2022 - Geocode - School/Shapefiles_SCH/EDGE_GEOCODE_PUBLICSCH_2122.shp'

In [3]:
# Get the available levels and student groups
available_levels = assessments['school_lvl'].unique()
available_groups = assessments['student_group'].unique().tolist()

In [4]:
# Define CSS Stylesheet
external_stylesheets = [dbc.themes.LUX]

In [5]:
# Define app with external scripts and stylesheets
app = JupyterDash(__name__,external_stylesheets=external_stylesheets)

In [6]:
# Build App
#app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

Application Layout

In [7]:
# App layout
app.layout = html.Div([
    html.H1("Achievement Testing Analysis", style={'text-align': 'center'}),
    html.Hr(),
    dcc.Markdown('''
        ## Welcome to my Capstone Project!
        ### 🤷‍♂️ I really don't know what I'm doing -and at this point I'm too afraid to ask. 🤷‍♂️ 
    '''),
    html.Hr(),
    html.Div([
        html.Div([
            html.H2("Proficiency Scores by School Level"),
            dcc.Markdown('''> This section will allow users to see the aggregated proficiency
            scores for all students at a given grade level.'''),
            dcc.Dropdown(
                id='school-level-dropdown',
                options=[
                    {'label': 'Elementary School', 'value': 'Elementary'},
                    {'label': 'Middle School', 'value': 'Secondary'},
                    {'label': 'High School', 'value': 'High'},
                    {'label': 'Other School Types', 'value': 'Other'},
                ],
                value='Elementary',
                clearable=False,
            ),
            dcc.Graph(id='graph1')
        ], className='graph-container'),
        html.Div([
            html.H2("Proficiency Scores By Student Group"),
            dcc.Markdown('''> This section will allow users to explore student groups that are associated
            with the school level they selected above.'''),
            dcc.Dropdown(
                id='student-group-dropdown',
                options=[
                    {'label': 'All Students', 'value': 'All Students'},
                    {'label': 'American Indian or Alaska Native', 'value': 'American Indian or Alaska Native'},
                    {'label': 'Asian', 'value': 'Asian'},
                    {'label': 'Black or African American', 'value': 'Black or African American'},
                    {'label': 'Black/Hispanic/Native American', 'value': 'Black/Hispanic/Native American'},
                    {'label': 'Economically Disadvantaged', 'value': 'Economically Disadvantaged'},
                    {'label': 'English Learner Transitional 1-4', 'value': 'English Learner Transitional 1-4'},
                    {'label': 'English Learners', 'value': 'English Learners'},
                    {'label': 'English Learners with Transitional 1-4', 'value': 'English Learners with Transitional 1-4'},
                    {'label': 'Female', 'value': 'Female'},
                    {'label': 'Gifted', 'value': 'Gifted'},
                    {'label': 'Hispanic', 'value': 'Hispanic'},
                    {'label': 'Male', 'value': 'Male'},
                    {'label': 'Native Hawaiian or Other Pacific Islander', 'value': 'Native Hawaiian or Other Pacific Islander'},
                    {'label': 'Non-Black/Hispanic/Native American', 'value': 'Non-Black/Hispanic/Native American'},
                    {'label': 'Non-Economically Disadvantaged', 'value': 'Non-Economically Disadvantaged'},
                    {'label': 'Non-English Learners/Transitional 1-4', 'value': 'Non-English Learners/Transitional 1-4'},
                    {'label': 'Non-Students with Disabilities', 'value': 'Non-Students with Disabilities'},
                    {'label': 'Students with Disabilities', 'value': 'Students with Disabilities'},
                    {'label': 'Super Subgroup', 'value': 'Super Subgroup'},
                    {'label': 'White', 'value': 'White'}
                ],
                value='Economically Disadvantaged',
                clearable=False,
            ),
            dcc.Graph(id='graph2')
        ], className='graph-container'),
        html.Div([
            html.H2("Proficiency Scores By Content Area and Year"),
            dcc.Markdown('''> This section will allow users to select a content area and year 
            and view a heatmap of proficiency scores on the map.'''),
            dcc.Dropdown(
                id='content-area-dropdown',
                options=[
                    {'label': 'English Language Arts', 'value': 'English Language Arts'},
                    {'label': 'Math', 'value': 'Math'},
                    {'label': 'Science', 'value': 'Science'},
                    {'label': 'Social Studies', 'value': 'Social Studies'},
                ],
                value='English Language Arts',
                clearable=False,
            ),
            dcc.Dropdown(
                id='year-dropdown',
                options=[
                    {'label': '2018', 'value': 2018},
                    {'label': '2019', 'value': 2019},
                    {'label': '2021', 'value': 2021},
                    {'label': '2022', 'value': 2022},
                ],
                value=2021,
                clearable=False,
            ),
            dcc.Graph(id='map')
        ], className='graph-container')
    ], className='flex-container'),
])

In [8]:
# Append local CSS file
app.css.append_css({'external_url': 'bWLwgP.css'})

Graph 1: School Level Proficiency by Subject

In [9]:
# Define callback to update graph1
@app.callback(
    Output('graph1', 'figure'),
    [Input('school-level-dropdown', 'value')]
)
def update_graph1(level):
    # Filter DataFrame based on selected level
    filtered_data = assessments[assessments['school_lvl'] == level]

    # Calculate average pct_met_exceeded for each subject_area and year
    average_data = filtered_data.groupby(['subject_area', 'year'])['pct_met_exceeded'].mean().reset_index()

    # Create a line chart
    fig = px.line(
        average_data,
        x='year',
        y='pct_met_exceeded',
        color='subject_area',
        title=f'{level} School: Proficiency by Subject',
        height=400,
        width=500,
    )

    # Update marker settings
    fig.update_traces(mode='markers+lines', marker=dict(size=5))

    # Update legends and axis labels
    fig.update_layout(
        legend_title='Content Area',
        xaxis_title='Year',
        yaxis_title=f'% Met or Exceeded'
    )

    return fig

Graph 2: Proficiency by Student Group

In [10]:
# Define callback to update graph2
@app.callback(
    Output('graph2', 'figure'),
    [Input('school-level-dropdown', 'value'),
     Input('student-group-dropdown', 'value')]
)
def update_graph2(level, group):
    # Filter DataFrame based on level and group selections
    if group == 'All Students':
        filtered_data = assessments[assessments['school_lvl'] == level]
        title = f'Another Chart for {level} Schools for All Students'
    else:
        filtered_data = assessments[(assessments['school_lvl'] == level) & (assessments['student_group'] == group)]
        title = f'{level} Level: {group} Group'

    # Calculate average pct_met_exceeded for each subject_area and year
    average_data = filtered_data.groupby(['subject_area', 'year'])['pct_met_exceeded'].mean().reset_index()

    # Create a line chart
    fig = px.line(
        average_data,
        x='year',
        y='pct_met_exceeded',
        color='subject_area',
        title=title,
        height=400,
        width=500,
    )

    # Update marker settings
    fig.update_traces(mode='markers+lines', marker=dict(symbol = 'circle', size=5))

    # Update legends and axis labels
    fig.update_layout(
        legend_title='Content Area',
        xaxis_title='Year',
        yaxis_title=f'% Met or Exceeded',
    )

    return fig

Map Visualization code

In [11]:
@app.callback(
    Output('map-container', 'children'),
    [Input('school-level-dropdown', 'value'),
     Input('student-group-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_map(school_level, student_group, year):
    filtered_data = assessments[(assessments['school_lvl'] == school_level) &
                                (assessments['subject_area'] == student_group) &
                                (assessments['year'] == year)]
    
    # Perform necessary data processing and aggregation
    # ...
    
    # Create choropleth map
    fig = px.choropleth_mapbox(
        filtered_data,
        geojson=counties,
        locations='fips',
        color='pct_met_exceeded',
        color_continuous_scale="Viridis",
        range_color=(0, 100),
        mapbox_style="carto-positron",
        zoom=3,
        center={"lat": 37.0902, "lon": -95.7129},
        opacity=0.5,
        labels={'pct_met_exceeded': 'Percent Met/Exceeded'}
    )
    
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    
    return dcc.Graph(id='map', figure=fig)

In [12]:
# Run the app
app.run_server(mode="jupyterlab")

Dash is running on http://127.0.0.1:8050/



/home/chunderdamus/anaconda3/lib/python3.9/site-packages/dash/resources.py:61: UserWarning: You have set your config to `serve_locally=True` but A local version of bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources
  warnings.warn(
